## Launch and set up a VM instance- with python-chi

We will use the `python-chi` Python API to Chameleon to provision our VM server.

We will execute the cells in this notebook inside the Chameleon Jupyter environment.

Run the following cell, and make sure the correct project is selected.

In [1]:
from chi import server, context
import chi, os, time, datetime

context.version = "1.0" 
context.choose_project()
context.choose_site(default="KVM@TACC")

MissingRequiredOptions: Auth plugin requires parameters which were not given: auth_url

We will use bring up a `m1.large` flavor server with the `CC-Ubuntu24.04` disk image.

> **Note**: the following cell brings up a server only if you don’t already have one with the same name! (Regardless of its error state.) If you have a server in ERROR state already, delete it first in the Horizon GUI before you run this cell.

In [2]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node-persist-{username}", 
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.large"
)
s.submit(idempotent=True)

Waiting for server node-persist-ahmed_offsechq_com's status to become ACTIVE. This typically takes 10 minutes for baremetal, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node-persist-ahmed_offsechq_com
Id,d3d203eb-62c5-4606-bf93-b47fe914996d
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,m1.large
Addresses,sharednet1: IP: 10.56.3.27 (v4) Type: fixed MAC: fa:16:3e:96:47:78
Network Name,sharednet1
Created At,2025-06-23T05:36:36Z
Keypair,ahmed_offsechq_com-jupyter
Reservation Id,None
Host Id,1960c1ebdd70db9cdd50edd47bad533c92358c78d794fa36c2a45920


In [11]:
4

4

Then, we’ll associate a floating IP with the instance:

In [3]:
s.associate_floating_ip()

'129.114.27.235'

In the output below, make a note of the floating IP that has been assigned to your instance (in the “Addresses” row).

In [4]:
s.refresh()
s.show(type="widget")

Attribute,node-persist-ahmed_offsechq_com
Id,d3d203eb-62c5-4606-bf93-b47fe914996d
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,m1.large
Addresses,sharednet1: IP: 10.56.3.27 (v4) Type: fixed MAC: fa:16:3e:96:47:78 IP: 129.114.27.235 (v4) Type: floating MAC: fa:16:3e:96:47:78
Network Name,sharednet1
Created At,2025-06-23T05:36:36Z
Keypair,ahmed_offsechq_com-jupyter
Reservation Id,None
Host Id,1960c1ebdd70db9cdd50edd47bad533c92358c78d794fa36c2a45920


By default, all connections to VM resources are blocked, as a security measure. We need to attach one or more “security groups” to our VM resource, to permit access over the Internet to specified ports.

The following security groups will be created (if they do not already exist in our project) and then added to our server:

In [5]:
security_groups = [
  {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
  {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"},
  {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used by MLFlow)"},
  {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used by MinIO API)"},
  {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used by MinIO Web UI)"}
]

In [6]:
chi.clients.connection()

In [7]:
# configure openstacksdk for actions unsupported by python-chi
os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

for sg in security_groups:

  if not os_conn.get_security_group(sg['name']):
      os_conn.create_security_group(sg['name'], sg['description'])
      os_conn.create_security_group_rule(sg['name'], port_range_min=sg['port'], port_range_max=sg['port'], protocol='tcp', remote_ip_prefix='0.0.0.0/0')

  nova_server.add_security_group(sg['name'])

print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

updated security groups: ['allow-8000', 'allow-8888', 'allow-9000', 'allow-9001', 'allow-ssh', 'default']


In [8]:
s.refresh()
s.check_connectivity()

Checking connectivity to 129.114.27.235 port 22.


Connection successful


### Retrieve code and notebooks on the instance

Now, we can use `python-chi` to execute commands on the instance, to set it up. We’ll start by retrieving the code and other materials on the instance.

In [9]:
s.execute("git clone https://github.com/teaching-on-testbeds/data-persist-chi")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 129.114.27.235: b'ac42305cdc52ca306ea8e62e93e9154f'
  warnings.warn(
Cloning into 'data-persist-chi'...


<Result cmd='git clone https://github.com/teaching-on-testbeds/data-persist-chi' exited=0>

### Set up Docker

Here, we will set up the container framework.

In [10]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c docker version


Client: Docker Engine - Community
 Version:           28.2.2
 API version:       1.50
 Go version:        go1.24.3
 Git commit:        e6534b4
 Built:             Fri May 30 12:07:27 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.2.2
  API version:      1.50 (minimum version 1.24)
  Go version:       go1.24.3
  Git commit:       45873be
  Built:            Fri May 30 12:07:27 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

<Result cmd='sudo groupadd -f docker; sudo usermod -aG docker $USER' exited=0>

In [28]:
# run in Chameleon Jupyter environment
from chi import context, server
import chi
import os

context.version = "1.0" 
context.choose_project()  # Select the correct project
context.choose_site(default="KVM@TACC")
username = os.getenv('USER') # exp resources will have this suffix


In [29]:
# run in Chameleon Jupyter environment
# delete the old server instance!
s_old = server.get_server(f"node-persist-{username}")
s_old.delete()

NotFound: Instance None could not be found. (HTTP 404) (Request-ID: req-70590a5d-8437-45d7-8b22-2d372b1906b9)

In [30]:
# run in Chameleon Jupyter environment
s = server.Server(
    f"node-persist-{username}", 
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.large"
)
s.submit(idempotent=True)

Waiting for server node-persist-ahmed_offsechq_com's status to become ACTIVE. This typically takes 10 minutes for baremetal, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node-persist-ahmed_offsechq_com
Id,1ba088e3-43aa-4992-9a63-4177a25418d7
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,m1.large
Addresses,sharednet1: IP: 10.56.1.249 (v4) Type: fixed MAC: fa:16:3e:cf:28:0d
Network Name,sharednet1
Created At,2025-06-23T09:49:38Z
Keypair,ahmed_offsechq_com-jupyter
Reservation Id,None
Host Id,1960c1ebdd70db9cdd50edd47bad533c92358c78d794fa36c2a45920


In [31]:
s.associate_floating_ip()

'129.114.27.235'

## Open an SSH session

Finally, open an SSH sesson on your server. From your local terminal, run

    ssh -i ~/.ssh/id_rsa_chameleon cc@A.B.C.D

where

-   in place of `~/.ssh/id_rsa_chameleon`, substitute the path to your own key that you had uploaded to KVM@TACC
-   in place of `A.B.C.D`, use the floating IP address you just associated to your instance.

In [33]:
s.refresh()
s.check_connectivity()

Checking connectivity to 129.114.27.235 port 22.


ResourceError: Waited too long for the port 22 on host 129.114.27.235 to start accepting connections.

In [17]:
# run in Chameleon Jupyter environment
s.refresh()
s.show(type="widget")

Attribute,node-persist-ahmed_offsechq_com
Id,6a178913-2fc2-4654-8ec1-4592caace6d3
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,m1.large
Addresses,sharednet1: IP: 10.56.2.13 (v4) Type: fixed MAC: fa:16:3e:dc:0c:3c IP: 129.114.27.235 (v4) Type: floating MAC: fa:16:3e:dc:0c:3c
Network Name,sharednet1
Created At,2025-06-23T08:30:39Z
Keypair,ahmed_offsechq_com-jupyter
Reservation Id,None
Host Id,1960c1ebdd70db9cdd50edd47bad533c92358c78d794fa36c2a45920


In [18]:
# run in Chameleon Jupyter environment
security_groups = [
  {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
  {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"},
  {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used by MLFlow)"},
  {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used by MinIO API)"},
  {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used by MinIO Web UI)"}
]

os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

for sg in security_groups:
  nova_server.add_security_group(sg['name'])

print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

updated security groups: ['allow-8000', 'allow-8888', 'allow-9000', 'allow-9001', 'allow-ssh', 'default']


In [ ]:
s.execute("git clone https://github.com/teaching-on-testbeds/data-persist-chi")

In [20]:
# run in Chameleon Jupyter environment
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c docker version


Client: Docker Engine - Community
 Version:           28.2.2
 API version:       1.50
 Go version:        go1.24.3
 Git commit:        e6534b4
 Built:             Fri May 30 12:07:27 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.2.2
  API version:      1.50 (minimum version 1.24)
  Go version:       go1.24.3
  Git commit:       45873be
  Built:            Fri May 30 12:07:27 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

<Result cmd='sudo groupadd -f docker; sudo usermod -aG docker $USER' exited=0>

In [27]:
# run in Chameleon Jupyter environment
cinder_client = chi.clients.cinder()
volume = [v for v in cinder_client.volumes.list() if v.name=='block-persist-netID'][0] # Substitute your own net ID

volume_manager = chi.nova().volumes
volume_manager.create_server_volume(server_id = s.id, volume_id = volume.id)

IndexError: list index out of range